In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
# Import Scikit-Learn library for decision tree models
import sklearn 

from sklearn import datasets, linear_model, metrics, tree
from sklearn.utils import resample

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor


from sklearn.neighbors import KNeighborsRegressor

        
# Dataset splitting        
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt


import os
import joblib

In [2]:
df = pd.read_csv("dataset_scaled.csv")
df

,Unnamed: 0,atomic_number,radius_pm,oxid_state,relativistic,metal_amount_mmol,num of group,spcaer,anchoring_group,chirality type,...,aromaticity1,sol1_vol_mL,protic2,polarity index 2,aromaticity2,solv2_vol_mL,time_min,temp_c,size_nm,g_factor*10^4
0,0,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,3.06,0
1,1,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.104117,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,1,0.757989206
2,2,-1.03686,-0.252283,-0.853143,-1.110555,1.601355,0.049297,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,1,0.58306862
3,3,-1.03686,-0.252283,-0.853143,-1.110555,1.768534,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.032487,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,2.37,0
4,4,0.90070,-0.626518,0.980887,0.900450,-0.454943,0.049297,-0.526910,-0.803389,0.120568,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,-1.03686,-0.252283,-0.853143,-1.110555,-0.265474,-0.115847,-0.742392,-0.803389,0.120568,...,-0.070711,-0.605526,-0.537086,-0.793018,-0.240613,-0.523310,0.217328,-0.176375,None,2
197,197,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.05,15
198,198,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.3,10
199,199,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.05,7


In [3]:
input_col = [col for col in df.columns]
input_col.remove('Unnamed: 0')
input_col.remove('size_nm')
input_col.remove('g_factor*10^4')
input_col

['atomic_number',
 'radius_pm',
 'oxid_state',
 'relativistic',
 'metal_amount_mmol',
 'num of group',
 'spcaer',
 'anchoring_group',
 'chirality type',
 'hydrogen bond',
 'aromaticity',
 'ligand_amount_mmol',
 'redu_num',
 'reductant_amount_mmol',
 'protic1',
 'polarity index1',
 'aromaticity1',
 'sol1_vol_mL',
 'protic2',
 'polarity index 2',
 'aromaticity2',
 'solv2_vol_mL',
 'time_min',
 'temp_c']

In [4]:
# Load ML model for predicting size

loaded_rf_size = joblib.load('model_aug_size_DecisionTree.joblib')

# Replace 'None' entries in 'size_nm' column by predicted values.

a = 0
for index, row in df.iterrows():
    if row['size_nm'] == 'None':
        X = df.loc[index, input_col].to_numpy()
        df.loc[index, 'size_nm'] = loaded_rf_size.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df.to_csv('dataset_augmented_size.csv')
df_first_aug = pd.read_csv('dataset_augmented_size.csv')

In [5]:
df_first_aug

,Unnamed: 0.1,Unnamed: 0,atomic_number,radius_pm,oxid_state,relativistic,metal_amount_mmol,num of group,spcaer,anchoring_group,...,aromaticity1,sol1_vol_mL,protic2,polarity index 2,aromaticity2,solv2_vol_mL,time_min,temp_c,size_nm,g_factor*10^4
0,0,0,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,3.060000,0
1,1,1,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,...,-0.070711,-0.104117,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,1.000000,0.757989206
2,2,2,-1.03686,-0.252283,-0.853143,-1.110555,1.601355,0.049297,-0.526910,-0.803389,...,-0.070711,-0.175747,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,1.000000,0.58306862
3,3,3,-1.03686,-0.252283,-0.853143,-1.110555,1.768534,-0.280991,-0.526910,-0.803389,...,-0.070711,-0.032487,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,2.370000,0
4,4,4,0.90070,-0.626518,0.980887,0.900450,-0.454943,0.049297,-0.526910,-0.803389,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,4.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,196,-1.03686,-0.252283,-0.853143,-1.110555,-0.265474,-0.115847,-0.742392,-0.803389,...,-0.070711,-0.605526,-0.537086,-0.793018,-0.240613,-0.523310,0.217328,-0.176375,2.066667,2
197,197,197,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.050000,15
198,198,198,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.300000,10
199,199,199,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.050000,7


In [6]:
#Saves the row indexes to drop in the column of g_factor into a list
total_row_num = len(df_first_aug)
drop_list_g =[]
for row_i in range(total_row_num):
    if df_first_aug['g_factor*10^4'].values[row_i] == 'None':
        drop_list_g.append(row_i)
    
len(drop_list_g)

#Drops the appropriate rows
df_g_scaled_encoded = df_first_aug.drop(drop_list_g)
#Saves the data for absorbance modelling to CSV
df_g_scaled_encoded.to_csv('dataset_scaled_g.csv')

## Move to 'g_aug_model' notebook to optimize model for augmenting Absorbance

In [7]:
# adding column 'size_nm' to input
input_col.append('size_nm')
input_col_2 = input_col.copy()
input_col.remove('size_nm')
print(input_col_2)

['atomic_number', 'radius_pm', 'oxid_state', 'relativistic', 'metal_amount_mmol', 'num of group', 'spcaer', 'anchoring_group', 'chirality type', 'hydrogen bond', 'aromaticity', 'ligand_amount_mmol', 'redu_num', 'reductant_amount_mmol', 'protic1', 'polarity index1', 'aromaticity1', 'sol1_vol_mL', 'protic2', 'polarity index 2', 'aromaticity2', 'solv2_vol_mL', 'time_min', 'temp_c', 'size_nm']


In [8]:
# Now, the dataset only has 'None' values in the 'Abs_nm' column

df_g = pd.read_csv('dataset_augmented_size.csv')

# Load ML model for predicting Aborbance

loaded_rf_g = joblib.load('model_aug_g_ExtraTrees.joblib')


# # Replace 'None' entries in 'Abs_nm' column by predicted values.

a = 0
for index, row in df_g.iterrows():
    if row['g_factor*10^4'] == 'None':
        X = df_g.loc[index, input_col_2].to_numpy()
        df_g.loc[index, 'g_factor*10^4'] = loaded_rf_g.predict(X.reshape(1, -1))[0]
        a += 1

# # # Save the dataset where all 'None' values are replaced.

# # # Ready to use for other analysis.

df_g.to_csv('dataset_augmented_all.csv')